In [ ]:
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import pickle
import gzip
import os.path
import importlib
import datetime

In [ ]:
from IPython.core.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

<h1> Fit data from beam based alignement measurement </h1>

<h2> Procedure </h2>

Measurements were made for the different quadrupoles using the multiplexer. These data have been preprocessed
BPM data were preprocessed in the followig manner:
* For each measurement step:
    * The first bpm data reading was discarded. This ensures that all 
      measurement data belong to the same machine setting.
    * The mean of the remaining data was calculated.
   
Based on the model beta(tron) functions are calculated to estimate the equivalent kick

In [ ]:
datetime.datetime.now().strftime('%Y %m %d %H:%M')

<h2> Used modules </h2>

The different modules are collected in :mod:`bact2`.

* The response matrix directory collects the different module
* :mod:`reference_orbit` try to provide an *side effect free* interface to 
  ocelot next to orbit difference processing
* :mod:`commons` provides access to the preprocessed data. The current solution is a hack and will have to be adapted to the available data bases

* the preprocessed data are currently stored in a pickle file. The pickle file is created using 
`from_json_to_pickle`

In [ ]:
import bact2.applib.bba.distorted_orbit_process
importlib.reload(bact2.applib.bba.distorted_orbit_process)
from bact2.applib.transverse_lib import reference_orbit, utils 
from bact2.applib.bba import distorted_orbit_process

importlib.reload(reference_orbit)
importlib.reload(distorted_orbit_process)


<h2> The model </h2>

The model is currently wrapped in :mod:`reference_orbit`. It allows

* calculating the reference orbit
* creating a new model with a changed element
* calculating offset from a changed orbit to the reference orbit without messing with the original model...

The following lines are used to set up the orbit and to store the reference data. Furthermore 
it is checked that an offset quadrupole is on orbit.

In [ ]:
default_orbit = reference_orbit.OrbitCalculator()

In [ ]:
moved_quadrupole_name = 'Q4M2D1R'

Check that the moved quadrupole found in the model does not expose an offset

In [ ]:
moved_quadrupole = default_orbit.getElementbyName(moved_quadrupole_name)
moved_quadrupole, moved_quadrupole.dx, moved_quadrupole.dy

In [ ]:
new_quad, orbit = default_orbit.orbitCalculatorWithNewElement(name=moved_quadrupole_name, init_lattice=False)

In [ ]:
new_quad, new_quad.dx, new_quad.dy

In [ ]:
new_quad.dx = 0
new_quad.dy = 0

In [ ]:
orbit.initLattice()

In [ ]:
orbit_data_ref = orbit.orbitData()

In [ ]:
orbit_offset_filter = reference_orbit.OrbitOffset()
orbit_offset_filter.reference_data = orbit_data_ref

<h2> Used data </h2>

The preprocessed data are available as a pickle file. The data are preprocessed by scripts provided by the `bact2` library.

In [ ]:
from bact2.applib.bba.process_dataframe import ProcessedBPMData
from bact2.applib.transverse_lib.from_json_to_pickle import preprocess_table

Defaults file. `bact.applib` apps assume that a directory 'data' exists on the same level as the bact2 repo directory. 
There it would expect the json dump next to intermediate pickle directores. 
This directory is not required for this notebook

In [ ]:
import bact2.applib.bba.commons as commons
pickle_file_name = commons.pickle_file_name()
json_file_name = commons.json_file_name()
pickle_file_name, json_file_name

In [ ]:
measured_data_df = pd.read_json(json_file_name)

It can be convienent to store the preprocessed data during development of the notebook

In [ ]:
use_pickle = False

In [ ]:
if not use_pickle:
    measured_data_pp_df = preprocess_table(measured_data_df)
    obj = ProcessedBPMData(measured_data_pp_df)

In [ ]:
if use_pickle:
    with gzip.open(pickle_file_name) as fp:
        obj = pickle.load(fp)

In [ ]:
obj.original_dataframe.head()

In [ ]:
obj.processed_dataframe.head()

<h2> Approximation function using equivalent kicker and  beta function </h2>

In [ ]:
twiss = orbit.twissParameters()
twiss_df = utils.twiss_to_df(twiss)
twiss_df.head()

In [ ]:
twiss_bpm = orbit.twissParametersBpms()
twiss_bpm_df = utils.twiss_to_df(twiss_bpm)
twiss_bpm_df.head()

In [ ]:
machine_x, machine_y = distorted_orbit_process.machine_info_xy(twiss_df)
machine_bpm_x, machine_bpm_y = distorted_orbit_process.machine_info_xy(twiss_bpm_df)
# Bpm data do not contain the whole machine .... 
machine_bpm_x.Q = machine_x.Q
machine_bpm_y.Q = machine_y.Q

In [ ]:
models = distorted_orbit_process.MachineModelXY(
    x=distorted_orbit_process.MachineModel(orbit=machine_x, bpm=machine_bpm_x, kick=None),
    y=distorted_orbit_process.MachineModel(orbit=machine_y, bpm=machine_bpm_y, kick=None)
)

<h2> Processing all  magnets </h2>
    
Currently assumes that all magnets found are quadrupoles magnets

In [ ]:
def kick_to_offset(angle, *, scale=1, k1=None):
    scale = float(scale)
    assert(k1 is not None)
    
    t_angle = angle * scale
    offset = t_angle / k1
    return offset

In [ ]:
importlib.reload(distorted_orbit_process)
muxed_magnets = set(obj.processed_dataframe.mux_selector_selected)
muxed_magnets = list(muxed_magnets)
columns_per_coordinate = ['kick', 'offset', 'fit_scale', 'guessed_angle']
columns = [n + '_x' for n in columns_per_coordinate]
columns += [n + '_y' for n in columns_per_coordinate]
columns += ['s', 'k1', 'tilt', 'dI', 'result']
rdf = pd.DataFrame(index=muxed_magnets, 
                   columns=columns,
                   dtype=np.object_
                  )

for t_name in tqdm.tqdm(muxed_magnets, total=len(muxed_magnets)):
    r = distorted_orbit_process.process_single(twiss_df, obj.processed_dataframe, t_name, models)
    elem = orbit.getElementbyName(t_name)
    rdf.at[t_name, 'k1'] = elem.k1
    rdf.at[t_name, 'tilt'] = elem.tilt
    rdf.at[t_name, 'result'] = r
    rdf.at[t_name, 's'] = r.x.kick.s
    rdf.at[t_name, 'fit_scale_x'] = r.x.fit_result.x[0]
    rdf.at[t_name, 'fit_scale_y'] = r.y.fit_result.x[0]
    rdf.at[t_name, 'dI'] = r.x.dI
    rdf.at[t_name, 'guessed_angle_x'] = r.x.guessed_angle
    rdf.at[t_name, 'guessed_angle_y'] = r.y.guessed_angle
rdf = rdf.sort_values(by='s')

In [ ]:
k1_ref = moved_quadrupole.k1 

In [ ]:
rdf.kick_x = rdf.guessed_angle_x * rdf.fit_scale_x
rdf.kick_y = rdf.guessed_angle_y * rdf.fit_scale_y
rdf.offset_x = kick_to_offset(rdf.kick_x, k1=rdf.k1 * 1 / 265)
rdf.offset_y = kick_to_offset(rdf.kick_y, k1=rdf.k1 * 1 / 265)

In [ ]:
rdf.head()

In [ ]:
fig = plt.figure(figsize=[16, 12])
ax1 = fig.add_subplot(211)
ax1.plot(rdf.s, rdf.offset_x * 1e6, 'b+-', linewidth=.5)
ax1.set_ylabel('x [$\mu$m]')
ax2 = fig.add_subplot(212)
ax2.plot(rdf.s, rdf.offset_y * 1e6, 'r+-', linewidth=.5)
ax2.set_ylabel('y [$\mu$m]')
ax2.set_xlabel('s [m]')

<h2> Reference plots for all magnets </h2>

In [ ]:
def create_plot_one_axis(ax, result, model, model_scale, dI, 
                         color=None, coordinate=None, magnet_name=None):
    
    assert(color is not None)
    assert(coordinate is not None)
    assert(np.isfinite(model_scale))

    co = result.orbit
    co_s = model.orbit.s
    co_bpm = result.bpm
    co_bpm_s = model.bpm.s
    
    p_scale_model = 1e6
    p_scale_data = 1e3
    # print(kick, dI.max())
    dI_max = dI.max()
    equivalent_angle = model_scale * dI_max
    p_scale_model_c = p_scale_model * equivalent_angle
    bpm_data = result.bpm_data
    bpm_offset = bpm_data[(0, -1), :].mean(axis=0)
    
    offset = result.fit_result.x[1]

    line, = ax.plot(co_s, (co + offset) * p_scale_model_c, '-', color=color, linewidth=.5)
    ax.plot(co_bpm_s, (co_bpm + offset) * p_scale_model_c, '.', color=line.get_color())
    
    scale = dI/dI.max()
    for i, do in enumerate(bpm_data):
        s = scale[i]
        if s == 0:
            s = 1
        ax.plot(co_bpm_s, (do - bpm_offset) * p_scale_data * s, '+--', color=line.get_color(), linewidth=.25)
    
    # Mark the position of the magnet
    magnet_s = result.kick.s
    axis = ax.axis()
    vmin = axis[2]
    vmax = axis[3]
    dv = vmax - vmin
    ax.plot([magnet_s] * 2, [vmin, vmax], 'k:', linewidth=2)
    ax.text(magnet_s, vmax - dv * 1/100., magnet_name, 
            horizontalalignment='left', verticalalignment='top')
    ax.axis(axis)
    ax.set_xlabel('s [m]')
    
    var_desc = r'$\Delta {}$'.format(coordinate)
    ax.set_ylabel(var_desc +  '  ' + r'[$\mu$m]')


def create_plots(rdf, models, fignum=10):
    
    fig = plt.figure(figsize=[16,12])
    ax = fig.add_subplot(211)
    magnet_name=rdf.name
    fit_scale_x = rdf.fit_scale_x
    fit_scale_y = rdf.fit_scale_y
    # fit_scale_x = fit_scale_y = 1
    
    kick = rdf.kick_x
    offset = rdf.offset_x
    fmt = '{} equivalent angle {:.3f} [$\mu$rad]  offset {:.1f} [$\mu$m]'
    txt = fmt.format(magnet_name, kick*1e6, offset*1e6)
    create_plot_one_axis(ax, rdf.result.x, models.x, fit_scale_x, rdf.dI, 
                         color='b', coordinate='x', magnet_name=txt)      
    ax = fig.add_subplot(212)
    kick = rdf.kick_y
    offset = rdf.offset_y
    txt = fmt.format(magnet_name, kick*1e6, offset*1e6)
    create_plot_one_axis(ax, rdf.result.y, models.y, fit_scale_y, rdf.dI, 
                        color='r', coordinate='y', magnet_name=txt)      
    return fig 

In [ ]:
#plt.ioff()
for idx, t_name in enumerate(rdf.index):
    plot_sel = rdf.loc[t_name, :]
    print(f'Magnet {t_name}, position {plot_sel.s:10.3f} fit: x {plot_sel.fit_scale_x: .3f} y {plot_sel.fit_scale_y: .3f}')
    #display(Markdown('<h3> Quadrupole {} </h4>'.format(t_name)))
    fig = create_plots(plot_sel, models=models)
    #fig.set_tight_layout(True)
    fig.savefig(f'BBA_Quadrupole_{t_name}.png')
    #break
    #del fig
    if idx >= 0:
        break
        pass

print('Done plotting')
#plt.ion()